In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G


# Loading Data

In [2]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_tic-tac-toe') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_1612/3852930321.py:1: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path('datasets.DecisionRuleSystems', 'DRS_tic-tac-toe') as dataset_path:
/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_1612/3852930321.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)


,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,class
0,NaN,NaN,NaN,NaN,o,o,NaN,o,o,positive
1,NaN,NaN,NaN,NaN,o,o,o,NaN,o,positive
2,NaN,NaN,NaN,NaN,o,o,o,o,NaN,positive
3,NaN,NaN,x,NaN,NaN,o,o,NaN,b,positive
4,NaN,x,NaN,x,NaN,NaN,b,NaN,b,positive
...,...,...,...,...,...,...,...,...,...,...
953,o,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,negative
954,NaN,NaN,o,x,NaN,NaN,NaN,NaN,NaN,negative
955,o,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,negative
956,o,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,negative


# Dataset Analyses

In [3]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

Number of features  = 9


In [4]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = False


In [5]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

d = 5


In [6]:
P = S
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AR = {len(B)}")

Length of Node Cover for AR = 5


In [7]:
P = R_SR(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for SR = {len(B)}")

Length of Node Cover for SR = 5


In [8]:
P = R_AD(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AD = {len(B)}")

Length of Node Cover for AD = 5


In [9]:
P = S
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AR = {len(B)}")

Length of Node Cover greedy for AR = 3


In [10]:
P = R_SR(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for SR = {len(B)}")

Length of Node Cover greedy for SR = 2


In [11]:
P = R_AD(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AD = {len(B)}")

Length of Node Cover greedy for AD = 2


# Combinations

In [12]:
column_values = []

for column in S.columns[:-1]: #ignoring class column
    column_values.append(list(S[column].dropna().unique()))
    
# All possible combinations    
combinations = list(itertools.product(*column_values))

print('Number of Possible combinations =', len(combinations))

Number of Possible combinations = 19683


In [13]:
ext_column_values = []

for column in S.columns[:-1]: #ignoring class column
    ext_column_values.append(list(S[column].dropna().unique()) + ['*'])
    
# All possible combinations with extension    
ext_combinations = list(itertools.product(*ext_column_values))

print('Number of Possible ext_combinations =', len(ext_combinations))

Number of Possible ext_combinations = 262144


# Node Cover

### AR

In [14]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [02:29<00:00, 131.38it/s]

NC AR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 9, 9.0, 149.83651685714722)


### EAR

In [15]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [32:15<00:00, 135.41it/s]

NC EAR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 9, 9.0, 1935.9245681762695)


### SR

In [16]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [33:58<00:00,  9.66it/s]

NC SR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 5, 8.22267946959305, 2038.4094789028168)


### ESR

In [17]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [7:27:23<00:00,  9.77it/s]

NC ESR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 5, 8.517105102539062, 26843.792709112167)


### AD

In [18]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [36:38<00:00,  8.95it/s]

NC AD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 6, 8.518670934308794, 2198.0995609760284)


### EAD

In [19]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [8:10:22<00:00,  8.91it/s]

NC EAD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 6, 8.636383056640625, 29422.80259990692)


# Node Cover Greedy

### AR

In [20]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [03:44<00:00, 87.60it/s]

NCgreedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 8, 8.992684042066758, 224.70013689994812)


### EAR

In [21]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [48:43<00:00, 89.67it/s]

NCgreedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 7, 8.899673461914062, 2923.3489401340485)


### SR

In [22]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [36:08<00:00,  9.08it/s]

NCgreedy SR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 5, 7.5868007925621095, 2168.243374109268)


### ESR

In [23]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [7:56:06<00:00,  9.18it/s]

NCgreedy ESR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 4, 7.77197265625, 28566.433598041534)


### AD

In [24]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [38:58<00:00,  8.42it/s]

NCgreedy AD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 6, 8.169791190367322, 2338.1777069568634)


### EAD

In [25]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [8:38:15<00:00,  8.43it/s]

NCgreedy EAD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 5, 8.02789306640625, 31095.7703332901)


# Greedy

### AR

In [26]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [01:42<00:00, 192.58it/s]

Greedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 8, 8.983539094650206, 102.21059036254883)


### EAR

In [27]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [21:36<00:00, 202.25it/s]

Greedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 7, 8.788375854492188, 1296.1209847927094)


### SR

In [28]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [34:59<00:00,  9.37it/s]

Greedy SR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 3, 6.341106538637403, 2099.5922498703003)


### ESR

In [29]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [7:45:34<00:00,  9.38it/s]

Greedy ESR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 3, 6.8431549072265625, 27934.420744895935)


### AD

In [30]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19683/19683 [38:22<00:00,  8.55it/s]

Greedy AD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 5, 7.390184423106234, 2302.882793903351)


### EAD

In [31]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [8:30:02<00:00,  8.57it/s]

Greedy EAD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 5, 7.3625335693359375, 30602.3201816082)
